In [1]:
import os
import torch
from torchvision import transforms

/home/robin/anaconda3/envs/text_extraction/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
! git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 1081, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1081 (delta 14), reused 73 (delta 11), pack-reused 998
Receiving objects: 100% (1081/1081), 69.97 MiB | 1.96 MiB/s, done.
Resolving deltas: 100% (479/479), done.


In [ ]:
%cd yolov7

In [9]:
! curl -L https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt -o yolov7-w6-pose.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  153M  100  153M    0     0  1958k      0  0:01:20  0:01:20 --:--:-- 1876k   0  1958k      0  0:01:20  0:00:41  0:00:39 2179k


In [ ]:
import torch
from torchvision import transforms

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
def load_model():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.load('/home/robin/Desktop/yolov7/yolo7_kpts_xtract/yolov7/yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [ ]:
def run_inference(url):
    image = cv2.imread(url) # shape: (480, 640, 3)
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (768, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 768, 960])
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 768, 960])
    output, _ = model(image) # torch.Size([1, 45900, 57])
    return output, image

In [9]:
def visualize_output(output, image):
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    
    
    
    for idx in range(output.shape[0]):
        #_skeleton_kpts(nimg, output[idx, 7:].T, 3)
        kpts = (output[idx,7:].T)
        new_kpt = [kpts]
        print(new_kpt)
#         landmarks = output[idx,7:].T
#         print("landmarks",landmarks)
#         print(len(landmarks))
#     plt.figure(figsize=(12, 12))
#     plt.axis('off')
#     plt.imshow(nimg)
#     plt.show()
    
#     cv2.imshow("res",nimg)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [10]:
output, image = run_inference('/home/robin/Desktop/assignment/images/images/Image_4.jpg') # Bryan Reyes on Unsplash
visualize_output(output, image)

[array([     429.83,      348.76,     0.99535,      421.84,       283.7,     0.99874,      381.69,      381.23,     0.97683,      518.47,       216.3,     0.91029,      398.81,      429.56,     0.17844,      729.24,      231.99,     0.55445,      566.29,      532.53,     0.75505,         853,      198.38,     0.16084,
            426.26,      553.01,     0.33805,      743.46,      173.06,     0.25613,      317.22,       376.1,     0.47167,      871.24,      287.07,    0.093104,      835.27,      449.38,      0.1367,      848.82,      300.15,    0.076528,      816.92,      490.51,     0.10875,      792.76,      437.29,    0.048305,
             747.1,       531.6,    0.060636])]


In [15]:
def ext_kpts(path):
    
    for f in os.listdir(path):
        img_path = path+"/"+f
        
        output,image = run_inference(img_path)
    
        output = non_max_suppression_kpt(output, 
                                         0.25, # Confidence Threshold
                                         0.65, # IoU Threshold
                                         nc=model.yaml['nc'], # Number of Classes
                                         nkpt=model.yaml['nkpt'], # Number of Keypoints
                                         kpt_label=True)
        with torch.no_grad():
            output = output_to_keypoint(output)
    
        all_kpts = []
        for idx in range(output.shape[0]):
            kpts = list(output[idx,7:].T)
            kpts.append(all_kpts)
        
        return all_kpts
        
    

In [16]:
ext_kpts("/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs")

[]

In [8]:
import os

path = "/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs"

for f in os.listdir(path):
    img_p = path+"/"+f
    print(img_p)

/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_37.jpg
/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_6.jpg
/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_33.jpg
/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_4.jpg
/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_15.jpg
/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_1.jpg


In [37]:
def draw_cir(image,kpts,draw = False):
    co_ord = []
    no_kpts = len(kpts)//3
    
    for i in range(no_kpts):
        cx,cy = kpts[3*i], kpts[3*i + 1]
        conf = kpts[3*i+2]
        co_ord.append([i,cx,cy,conf])
        cv2.circle(image, (cx,cy),10,(255,0,0),cv2.FILLED)
    
    print("conf", conf)
    print("co_ord",co_ord)
        
    cv2.imshow("res",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [10]:
def pose_video(frame):
    #mapped_img = frame.copy()
    # Letterbox resizing.
    img = letterbox(frame, input_size, stride=64, auto=True)[0]
    print(img.shape)
    img_ = img.copy()
    # Convert the array to 4D.
    img = transforms.ToTensor()(img)
    # Convert the array to Tensor.
    img = torch.tensor(np.array([img.numpy()]))
    # Load the image into the computation device.
    img = img.to(device)
     
    # Gradients are stored during training, not required while inference.
    with torch.no_grad():
        t1 = time.time()
        output, _ = model(img)
        t2 = time.time()
        fps = 1/(t2 - t1)
        output = non_max_suppression_kpt(output, 
                                         0.25,    # Conf. Threshold.
                                         0.65,    # IoU Threshold.
                                         nc=1,   # Number of classes.
                                         nkpt=17, # Number of keypoints.
                                         kpt_label=True)
         
        output = output_to_keypoint(output)
 
    # Change format [b, c, h, w] to [h, w, c] for displaying the image.
    nimg = img[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
 
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
         
    return nimg, fps

In [18]:
#pose_video("'/home/robin/Desktop/assignment/images/images/Image_362.jpg'")

In [17]:
import pandas as pd
df = pd.DataFrame(columns=["Name","KPTS"])

In [18]:
df

,Name,KPTS


In [25]:
path = "/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs"

for f in os.listdir(path):
    print(f)
    #img_p = path+"/"+f
    
    df["Name"] = f

Image_37.jpg
Image_6.jpg
Image_33.jpg
Image_4.jpg
Image_15.jpg
Image_1.jpg


In [24]:
df

,Name,KPTS


In [18]:
dff = pd.read_csv("/home/robin/Desktop/yolov7/yolo7_kpts_xtract/labels.csv",header = None)
dff

,0,1
0,Image_1.jpg,sitting
1,Image_4.jpg,sleeping
2,Image_6.jpg,sleeping
3,Image_15.jpg,sleeping
4,Image_33.jpg,sitting
...,...,...
1264,Image_12527.jpg,running
1265,Image_12577.jpg,sleeping
1266,Image_12594.jpg,sleeping
1267,Image_12596.jpg,sitting


In [19]:
dff = dff.rename(columns={0: 'Name', 1: 'Label'})

In [21]:
dff[:3]

,Name,Label
0,Image_1.jpg,sitting
1,Image_4.jpg,sleeping
2,Image_6.jpg,sleeping


In [34]:
len(dff)

1269

In [ ]:
for i in range(10):
    path = "/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs"
    

In [46]:
filenames = ["/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/" + fname for fname in dff["Name"]]


In [47]:
filenames[:5]

['/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_1.jpg',
 '/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_4.jpg',
 '/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_6.jpg',
 '/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_15.jpg',
 '/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_33.jpg']

In [16]:
test_path = "/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/"
test_filename = [test_path + fname for fname in os.listdir(test_path)]

In [23]:
test_filename

['/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_6.jpg',
 '/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_4.jpg',
 '/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs/Image_1.jpg']

In [31]:
def ext_kpts(path):
    
    all_kpt = []
    
    
    for f in os.listdir(path):
        img_p = path+"/"+f
    
        output,image = run_inference(img_p)

        output = non_max_suppression_kpt(output, 
                                         0.25, # Confidence Threshold
                                         0.65, # IoU Threshold
                                         nc=model.yaml['nc'], # Number of Classes
                                         nkpt=model.yaml['nkpt'], # Number of Keypoints
                                         kpt_label=True)
        with torch.no_grad():
            output = output_to_keypoint(output)


        for idx in range(output.shape[0]):
            kpts = (output[idx,7:].T)
            new_kpts = [i for i in kpts]
            new_kpts.append(all_kpt)
            kpts = []
            
    return all_kpt
        
    

In [32]:
path = "/home/robin/Desktop/yolov7/yolo7_kpts_xtract/imgs"

In [33]:
ext_kpts(path)

[]

In [10]:
all_kpt

NameError: name 'all_kpt' is not defined

In [22]:
#df.drop('KPTS', inplace=True, axis=1)

In [23]:
df

,Name,KPTS
0,NaN,"[358.8403625488281, 154.9840087890625, 0.99754..."


In [24]:
df["KPTS"]

0    [358.8403625488281, 154.9840087890625, 0.99754...
Name: KPTS, dtype: object